# Generate Embeddings on Google Colab

This notebook runs embedding generation on Google Colab's free GPU.

**Steps:**
1. Upload your dataset CSV file
2. Install dependencies
3. Run embedding generation
4. Download results


## Step 1: Check GPU Availability


In [ ]:
# Check if GPU is available
import torch

if torch.cuda.is_available():
    print(f"✅ GPU Available: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    device = "cuda"
else:
    print("❌ No GPU available. Please enable GPU in Runtime > Change runtime type > GPU")
    device = "cpu"


## Step 2: Install Dependencies


In [ ]:
# Install required packages
%pip install sentence-transformers pandas numpy tqdm -q


## Step 3: Upload Dataset

Upload your `standards_dataset.csv` file using the file uploader below.


In [ ]:
from google.colab import files
import pandas as pd
from pathlib import Path
import os

# Create datasets directory
os.makedirs("datasets", exist_ok=True)
os.makedirs("datasets/embeddings", exist_ok=True)

print("Please upload your standards_dataset.csv file:")
uploaded = files.upload()

# Move uploaded file to datasets directory
dataset_path = None
for filename in uploaded.keys():
    if filename.endswith('.csv'):
        os.rename(filename, f"datasets/{filename}")
        print(f"✅ Uploaded: {filename}")
        dataset_path = f"datasets/{filename}"
        break

if not dataset_path:
    print("❌ No CSV file found. Please upload standards_dataset.csv")
else:
    # Verify dataset
    df = pd.read_csv(dataset_path)
    print(f"\nDataset loaded: {len(df)} sections")
    print(f"Columns: {list(df.columns)}")


## Step 4: Generate Embeddings


In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import json
import time

if 'dataset_path' not in locals() or dataset_path is None:
    print("❌ Please upload dataset first!")
else:
    # Load dataset
    df = pd.read_csv(dataset_path)
    texts = df['content'].tolist()
    
    print(f"Total texts to embed: {len(texts):,}")
    
    # Model configuration
    model_name = "BAAI/bge-m3"  # High quality model
    batch_size = 256 if device == "cuda" else 16
    use_fp16 = device == "cuda"
    
    print(f"\nConfiguration:")
    print(f"  Model: {model_name}")
    print(f"  Device: {device}")
    print(f"  Batch size: {batch_size}")
    print(f"  FP16: {use_fp16}")
    
    # Load model
    print(f"\nLoading model...")
    model = SentenceTransformer(model_name, device=device)
    if use_fp16:
        model = model.half()
        print("Using FP16 (half precision) for faster processing")
    
    # Generate embeddings
    print(f"\nGenerating embeddings...")
    total_batches = (len(texts) + batch_size - 1) // batch_size
    embeddings = []
    start_time = time.time()
    
    with tqdm(total=len(texts), desc="Processing", unit="text") as pbar:
        for batch_idx, i in enumerate(range(0, len(texts), batch_size), 1):
            batch = texts[i:i+batch_size]
            
            pbar.set_description(f"Batch {batch_idx}/{total_batches}")
            
            batch_embeddings = model.encode(
                batch,
                show_progress_bar=False,
                convert_to_numpy=True,
                normalize_embeddings=True,
                batch_size=len(batch)
            )
            
            embeddings.append(batch_embeddings)
            pbar.update(len(batch))
            
            # Show progress
            elapsed = time.time() - start_time
            texts_processed = min(i + len(batch), len(texts))
            texts_per_sec = texts_processed / elapsed if elapsed > 0 else 0
            
            if batch_idx > 0 and elapsed > 0:
                avg_time_per_text = elapsed / texts_processed
                remaining_texts = len(texts) - texts_processed
                eta = remaining_texts * avg_time_per_text
            else:
                eta = 0
            
            pbar.set_postfix({
                'Speed': f"{texts_per_sec:.1f} texts/s",
                'ETA': f"{eta:.0f}s" if eta > 0 else "calculating..."
            })
    
    # Concatenate embeddings
    all_embeddings = np.vstack(embeddings)
    total_time = time.time() - start_time
    
    print(f"\n{'='*60}")
    print(f"✅ Embeddings generated successfully!")
    print(f"{'='*60}")
    print(f"Total time: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")
    print(f"Average speed: {len(texts)/total_time:.1f} texts/second")
    print(f"Embedding shape: {all_embeddings.shape}")
    print(f"Embedding dimension: {all_embeddings.shape[1]}")
    
    # Save embeddings
    embeddings_path = "datasets/embeddings/embeddings.npy"
    np.save(embeddings_path, all_embeddings)
    print(f"\n✅ Saved embeddings to: {embeddings_path}")
    
    # Save metadata
    metadata = {
        'model_name': model_name,
        'embedding_dimension': int(all_embeddings.shape[1]),
        'num_embeddings': int(all_embeddings.shape[0]),
        'normalized': True,
        'processing_time_seconds': float(total_time),
        'texts_per_second': float(len(texts) / total_time)
    }
    
    metadata_path = "datasets/embeddings/metadata.json"
    with open(metadata_path, 'w') as f:
        json.dump(metadata, f, indent=2)
    print(f"✅ Saved metadata to: {metadata_path}")
    
    # Update dataset with embedding indices
    df['embedding_index'] = range(len(df))
    output_dataset = "datasets/standards_dataset_with_embeddings.csv"
    df.to_csv(output_dataset, index=False)
    print(f"✅ Saved dataset with indices to: {output_dataset}")


## Step 5: Download Results


In [ ]:
from google.colab import files
import zipfile
import os

# Create zip file with all results
zip_path = "embeddings_results.zip"
with zipfile.ZipFile(zip_path, 'w') as zipf:
    # Add embeddings
    if os.path.exists("datasets/embeddings/embeddings.npy"):
        zipf.write("datasets/embeddings/embeddings.npy", "embeddings.npy")
    
    # Add metadata
    if os.path.exists("datasets/embeddings/metadata.json"):
        zipf.write("datasets/embeddings/metadata.json", "metadata.json")
    
    # Add dataset with indices
    if os.path.exists("datasets/standards_dataset_with_embeddings.csv"):
        zipf.write("datasets/standards_dataset_with_embeddings.csv", "standards_dataset_with_embeddings.csv")

print("📦 Created zip file with all results")
print("\nDownloading files...")
files.download(zip_path)
print("\n✅ Download complete!")
print("\nFiles included:")
print("  - embeddings.npy (embeddings array)")
print("  - metadata.json (model info)")
print("  - standards_dataset_with_embeddings.csv (dataset with indices)")
